In [1]:
import numpy as np
import pandas as pd

# Uncomment to see full cell text
pd.set_option('display.max_colwidth', 0)

faq = pd.read_csv('../data/interim/faq-text-separated.csv', keep_default_na=False)

# NLP NLTK

In [2]:
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk 
import nltk.data
nltk.download('punkt')
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
#nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/willscott/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Sentence Tokenizing

In [31]:
def preprocess(entry):
    entry = entry.replace('\n', ' ')
    entry = unicodedata.normalize("NFKD", entry)
    entry = entry.replace('?', '? ')
#     re.sub(r'.(?=[A-Z])', '. ', entry)
    return entry + ' '

raw = pd.read_excel('../data/raw/inquire-boulder-faq-text/inquire-boulder-active-faqs-2019-01-02.xls.xlsx')
raw = raw.drop(labels=('Active'), axis=1).dropna()
raw.FAQ = raw.FAQ.apply(preprocess)

In [32]:
raw

Topic  \
0    Affordable Homeownership                                           
1    Affordable Rental Opportunities                                    
2    Aircraft Noise                                                     
3    Airport                                                            
4    Alley Maintenance                                                  
5    Americans for the Arts Convention Scholarships                     
6    Americans with Disabilities Act (ADA)                              
7    Americans with Disabilities Act (ADA) Trails                       
8    Animal Noise (Barking Dogs)                                        
9    Animal Protection, Rules and Regulations                           
10   Animal Shelter                                                     
11   Ask-a-Librarian                                                    
12   Auction Licensing Instructions                                     
13   Backflow Prevention - Drinking Water                               
14   Bear Resistant Container Required                                  
15   Bears                                                              
16   Bicycling                                                          
17   Birth and Death Certificate Request                                
18   Block Party                                                        
19   Boulder Arts Commission Grants                                     
20   Boulder City Limits                                                
21   Boulder Civic Area                                                 
22   Boulder Convention and Visitors Bureau                             
23   Boulder Measures                                                   
24   Boulder Office of Emergency Management                             
25   Boulder Planning                                                   
26   Boulder Reservoir                                                  
27   Boulder Revised Code                                               
28   BoulderReads!                                                      
29   Broken Parking Meter or Pay Station                                
..                                   ...                                
223  Theater Rental and Marketing Assistance Grants                     
224  Towed/Impounded Vehicles                                           
225  Traffic Signals                                                    
226  Traffic Signs and Markings                                         
227  Train Noise and Quiet Zones                                        
228  Transit (Bus Service)                                              
229  Transportation Flood Recovery                                      
230  Transportation Projects                                            
231  Trash Concerns                                                     
232  Tree Branch Pick-Up                                                
233  Twenty Ninth Street                                                
235  Visitors                                                           
236  Volunteer Opportunities                                            
237  Volunteering at the Library                                        
238  Wage Issues                                                        
239  Waste Hauling Services                                             
240  Water Being Pumped or Diverted Off Private Property to Elsewhere   
241  Water Seepage                                                      
242  Water System Maintenance                                           
243  Water Utility Billing                                              
244  Water, Wastewater, Stormwater/Drainageway Flood Recovery           
245  Website Issues                                                     
246  Weed Control and Tall Vegetation                                   
247  Wildlife Issue or Sighting                  

In [26]:
# Built-in sentence tokenizer
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [60]:
print('\n-----\n'.join(sent_detector.tokenize(raw.FAQ[1].strip())))

The City of Boulder is committed to maintaining a variety of rental housing options that are affordable to residents of all income levels.
-----
Low-income/subsidized rentals Boulder Housing Partners (720-564-4600) Thistle Community Housing (303-443-0007) Boulder County Housing Authority (720-564-2284) Senior rentals and other housingSenior Housing Guide for Boulder County Pets in rental housingHumane Society of Boulder Valley List of Pet-friendly Rental Properties Rental Licensing The City of Boulder's Rental Housing Inspection and Licensing Program exists to preserve public safety and establishes minimum standards for basic equipment and facilities for rental property.
-----
For more information, see: Rental Licensing Renter's rights (Landlord/tenant information and mediation) For rental properties located within Boulder city limits, contact Community Mediation Services at 303-441-4364.
-----
For University of Colorado students, call Off-Campus Housing & Neighborhood Relations at 303

In [61]:
# Training custom sentence tokenizer
# Looks like it is a slight improvement! ('2.' is not isolated as a sentence in entry 0)
custom_tokenizer = PunktSentenceTokenizer(str(raw.FAQ))
print('\n-----\n'.join(custom_tokenizer.tokenize(raw.FAQ[1])))

The City of Boulder is committed to maintaining a variety of rental housing options that are affordable to residents of all income levels.
-----
Low-income/subsidized rentals Boulder Housing Partners (720-564-4600) Thistle Community Housing (303-443-0007) Boulder County Housing Authority (720-564-2284) Senior rentals and other housingSenior Housing Guide for Boulder County Pets in rental housingHumane Society of Boulder Valley List of Pet-friendly Rental Properties Rental Licensing The City of Boulder's Rental Housing Inspection and Licensing Program exists to preserve public safety and establishes minimum standards for basic equipment and facilities for rental property.
-----
For more information, see: Rental Licensing Renter's rights (Landlord/tenant information and mediation) For rental properties located within Boulder city limits, contact Community Mediation Services at 303-441-4364.
-----
For University of Colorado students, call Off-Campus Housing & Neighborhood Relations at 303

# Stemming

In [87]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

porter=PorterStemmer()

def stemSentence(sentence):
    sentence = " ".join(sentence)
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
    return stem_sentence

def text_process(text):
    # Remove punctuation
    text = text.replace('-', ' ')
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    nopunc = [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    nopunc = stemSentence(nopunc)
    
    return nopunc

In [88]:
text_process('# Vectorization with Bag of Words')

['vector', 'bag', 'word']

# Lemmatization

In [7]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()

def lemSentence(sentence):
    sentence = " ".join(sentence)
    token_words=word_tokenize(sentence)
    token_words
    lem_sentence=[]
    for word in token_words:
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word))
    return lem_sentence

def text_process(text):
    # Remove punctuation
    text = text.replace('-', ' ')
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    # Remove stopwords
    nopunc = [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    nopunc = lemSentence(nopunc)
    
    return nopunc

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/willscott/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [8]:
text_process('# Vectorization with Bag of Words')

['vectorization', 'bag', 'word']

# Vectorization with Bag of Words

In [9]:
faq['answer'].apply(text_process)

0      [generally, two, week]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
1      [everyone, life, household, part, household, addition, people, listed, property, title, hous

In [10]:
# Might take a while...
bow_transformer = CountVectorizer(analyzer=text_process).fit(faq.question)

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

1088


In [11]:
# Let's take onequestion and get its bag-of-words counts as a vector, putting to use our new `bow_transformer`:
entry = faq['question'][0]
print(entry)

How long does it take to become income-certified? 


In [12]:
# Now let's see its vector representation:
bow = bow_transformer.transform([entry])
print(bow)
print(bow.shape)

  (0, 96)	1
  (0, 164)	1
  (0, 486)	1
  (0, 571)	1
  (0, 985)	1
(1, 1088)


This means that there are X unique words in message (after removing common stop words). 

In [13]:
print(bow_transformer.get_feature_names()[867])

right


Now we can use .transform on our Bag-of-Words (bow) transformed object and transform the entire DataFrame. Let's go ahead and check out how the bag-of-words counts for the entire FAQ corpus is a large, sparse matrix:

In [14]:
faq_bow = bow_transformer.transform(faq['question'])
print('Shape of Sparse Matrix: ', faq_bow.shape)
print('Amount of Non-Zero occurences: ', faq_bow.nnz)

Shape of Sparse Matrix:  (649, 1088)
Amount of Non-Zero occurences:  2842


In [15]:
sparsity = (100.0 * faq_bow.nnz / (faq_bow.shape[0] * faq_bow.shape[1]))
print('sparsity: {}% Non-zero values'.format(round(sparsity, ndigits=2)))

sparsity: 0.4% Non-zero values


In [16]:
tfidf_transformer = TfidfTransformer().fit(faq_bow)
tfidf = tfidf_transformer.transform(bow)
print(tfidf)

  (0, 985)	0.3771095145034031
  (0, 571)	0.40075395419269094
  (0, 486)	0.4356506169713436
  (0, 164)	0.5036830462713718
  (0, 96)	0.5036830462713718


In [18]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['boulder']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['county']])

3.287317620863257
6.090678001769792


In [19]:
faq_tfidf = tfidf_transformer.transform(faq_bow)
print(faq_tfidf.shape)

(649, 1088)


### Train the model

In [20]:
from sklearn.naive_bayes import MultinomialNB
faq_match_model = MultinomialNB().fit(faq_tfidf, faq['Topic'])

In [21]:
print('predicted:', faq_match_model.predict(tfidf)[0])
print('expected:', faq.Topic[0])

predicted: Affordable Homeownership
expected: Affordable Homeownership


### Evaluate the model

In [22]:
pred = faq_match_model.predict(faq_tfidf)
print(pred)

['Affordable Homeownership'
 'Hourly Billing Charges for Development Review Applications'
 'Affordable Homeownership' 'Affordable Homeownership'
 'Affordable Homeownership' 'SmartRegs' 'SmartRegs' 'SmartRegs'
 'Connect Boulder Wi-Fi' 'Current Affordable Homeowners' 'SmartRegs'
 'SmartRegs' 'Airport' 'Airport' 'Airport' 'Parking Tickets' 'Airport'
 'Airport' 'SmartRegs' 'Airport' 'Airport' 'Parking Tickets' 'SmartRegs'
 'SmartRegs' 'SmartRegs' 'Prairie Dogs' 'Prairie Dogs' 'SmartRegs'
 'Park Shelter Rental' 'SmartRegs' 'SmartRegs' 'SmartRegs' 'Bears'
 'SmartRegs' 'Bear Resistant Container Required' 'Parking Tickets'
 'Building Permits' 'Bear Resistant Container Required' 'Parking Tickets'
 'Parking Tickets' 'Bears' 'Bears' 'Bears' 'Bears' 'Bears'
 'Mountain Lions' 'Bears' 'Bears' 'Bears' 'Bears' 'Bears' 'Bears' 'Bears'
 'Bears' 'Bears' 'SmartRegs' 'Parking Tickets' 'Bicycling' 'Bicycling'
 'Parking Tickets' 'Parking Tickets' 'SmartRegs' 'SmartRegs' 'SmartRegs'
 'Park Shelter Rental' 'Sm

In [23]:
from sklearn.metrics import classification_report
print (classification_report(faq['Topic'], pred))

                                                                      precision    recall  f1-score   support

                                                  Accessible Parking       0.00      0.00      0.00         1
                                            Affordable Homeownership       1.00      0.40      0.57        10
                                     Affordable Rental Opportunities       0.00      0.00      0.00         1
                                                      Aircraft Noise       0.00      0.00      0.00         1
                                                             Airport       1.00      0.78      0.88         9
                                                   Alley Maintenance       0.00      0.00      0.00         1
                      Americans for the Arts Convention Scholarships       0.00      0.00      0.00         1
                               Americans with Disabilities Act (ADA)       0.00      0.00      0.00         1
         

/Users/willscott/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
from sklearn.model_selection import train_test_split

faq_train, faq_test, topic_train, topic_test = \
train_test_split(faq['question'], faq['Topic'], test_size=0.2)

print(len(faq_train), len(faq_test), len(faq_train) + len(faq_test))

519 130 649


In [25]:
from sklearn.pipeline import Pipeline

tfidf_transformer = TfidfTransformer().fit(faq_bow)

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', tfidf_transformer),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [26]:
pipeline.fit(faq_train,topic_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x1a0b0df488>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [27]:
pred = pipeline.predict(faq_test)

In [28]:
print(classification_report(pred, faq_test))

                                                                                                                                                                                                                                              precision    recall  f1-score   support

                                                                                                                                                                                              Americans for the Arts Convention Scholarships       0.00      0.00      0.00         0
                                                                                                                                                                                                Americans with Disabilities Act (ADA) Trails       0.00      0.00      0.00         0
                                                                                                                                                                     

/Users/willscott/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/willscott/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
